# K7 G2 Metric Validation - Publication Quality

**GIFT Framework v2.2** - Complete validation of G2 holonomy metric on K7 manifold.

This notebook provides rigorous validation of:
1. **Metric invariants**: det(g) = 65/32, kappa_T = 1/61
2. **G2 structure**: Exact and approximate metric reconstruction from phi
3. **Betti numbers**: b2 = 21, b3 = 77 (35 local + 42 global)
4. **Yukawa tensor**: SM-like flavor structure emergence

---

In [ ]:
import sys
import torch
import numpy as np
import matplotlib.pyplot as plt

# Add project root to path
sys.path.insert(0, '../..')

from G2_ML.meta_hodge.geometry_loader import (
    load_version_model,
    locate_historical_assets,
    verify_g2_identity,
    phi_to_metric_exact,
    phi_to_metric_approximate,
    canonical_g2_phi,
    calibrate_phi_for_det,
    levi_civita_7d,
)

# GIFT constants
DET_G_TARGET = 65.0 / 32.0   # 2.03125
KAPPA_T_TARGET = 1.0 / 61.0  # 0.016393
B2_TARGET = 21
B3_TARGET = 77

print(f"GIFT Structural Constants:")
print(f"  det(g) = 65/32 = {DET_G_TARGET:.6f}")
print(f"  kappa_T = 1/61 = {KAPPA_T_TARGET:.6f}")
print(f"  b2 = {B2_TARGET}, b3 = {B3_TARGET}")

## 1. Load v1.6 GIFT-Calibrated Data

In [ ]:
# Load v1.6 pre-computed data
registry = locate_historical_assets()
bundle = load_version_model('1_6', registry)
print(f"Bundle notes: {bundle.notes}")

precomputed = bundle.precomputed
if precomputed is None:
    raise ValueError("No precomputed data found for v1.6")

x = precomputed.x
phi_tensor = precomputed.phi_tensor
g = precomputed.g
det_g = precomputed.det_g
torsion = precomputed.torsion

N = x.shape[0]
print(f"\nData loaded: {N} samples")
print(f"  x shape: {x.shape}")
print(f"  phi shape: {phi_tensor.shape}")
print(f"  g shape: {g.shape}")

## 2. Metric Invariant Validation

In [ ]:
print("=" * 60)
print("METRIC INVARIANT VALIDATION")
print("=" * 60)

# det(g) check
det_g_mean = det_g.mean().item()
det_g_std = det_g.std().item()
det_g_dev = abs(det_g_mean - DET_G_TARGET) / DET_G_TARGET * 100

print(f"\ndet(g):")
print(f"  Target:    {DET_G_TARGET:.6f}")
print(f"  Measured:  {det_g_mean:.6f} +/- {det_g_std:.6f}")
print(f"  Deviation: {det_g_dev:.4f}%")
print(f"  Status:    {'EXACT' if det_g_dev < 0.01 else 'PASS' if det_g_dev < 1 else 'FAIL'}")

# kappa_T check
if torsion is not None:
    kappa_mean = torsion.mean().item()
    kappa_std = torsion.std().item()
    kappa_dev = abs(kappa_mean - KAPPA_T_TARGET) / KAPPA_T_TARGET * 100
    
    print(f"\nkappa_T:")
    print(f"  Target:    {KAPPA_T_TARGET:.6f}")
    print(f"  Measured:  {kappa_mean:.6f} +/- {kappa_std:.6f}")
    print(f"  Deviation: {kappa_dev:.2f}%")
    print(f"  Status:    {'PASS' if kappa_dev < 1 else 'WARN' if kappa_dev < 5 else 'FAIL'}")
else:
    print("\nkappa_T: Not available in precomputed data")

In [ ]:
# Visualize det(g) distribution
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Histogram
axes[0].hist(det_g.numpy(), bins=50, density=True, alpha=0.7, color='steelblue')
axes[0].axvline(DET_G_TARGET, color='red', linestyle='--', linewidth=2, label=f'Target = {DET_G_TARGET:.4f}')
axes[0].axvline(det_g_mean, color='green', linestyle='-', linewidth=2, label=f'Mean = {det_g_mean:.4f}')
axes[0].set_xlabel('det(g)')
axes[0].set_ylabel('Density')
axes[0].set_title('det(g) Distribution')
axes[0].legend()

# Deviation from target
deviation = ((det_g - DET_G_TARGET) / DET_G_TARGET * 100).numpy()
axes[1].hist(deviation, bins=50, density=True, alpha=0.7, color='coral')
axes[1].axvline(0, color='red', linestyle='--', linewidth=2)
axes[1].set_xlabel('Relative Deviation (%)')
axes[1].set_ylabel('Density')
axes[1].set_title('det(g) Deviation from Target')

plt.tight_layout()
plt.show()

## 3. G2 Structure Validation

In [ ]:
print("=" * 60)
print("G2 STRUCTURE VALIDATION")
print("=" * 60)

# Use subset for faster computation
n_test = min(500, N)
phi_test = phi_tensor[:n_test]
g_test = g[:n_test]

# Verify G2 identities
results = verify_g2_identity(phi_test, g_test)

print(f"\n||phi||^2_g (G2 norm):")
norm_sq = results['phi_norm_sq']
print(f"  Mean: {norm_sq.mean():.4f}")
print(f"  Std:  {norm_sq.std():.4f}")
print(f"  Note: For canonical G2, target is 7.0 with standard normalization")

print(f"\nMetric Reconstruction Deviation:")
print(f"  Approximate formula: max|g - g_approx| = {results['g_deviation_approx'].mean():.4f}")
print(f"  Exact formula:       max|g - g_exact|  = {results['g_deviation_exact'].mean():.4f}")

In [ ]:
# Compare exact vs approximate formulas
g_approx = results['g_reconstructed_approx']
g_exact = results['g_reconstructed_exact']

det_approx = torch.linalg.det(g_approx)
det_exact = torch.linalg.det(g_exact)

print(f"\ndet(g) from phi reconstruction:")
print(f"  Original (v1.6):   {det_g[:n_test].mean():.6f}")
print(f"  Approx formula:    {det_approx.mean():.6f}")
print(f"  Exact formula:     {det_exact.mean():.6f}")

print(f"\nNote: v1.6 metric was learned directly, not via phi -> g reconstruction.")
print(f"      The stored phi represents G2 structure but uses different normalization.")

## 4. Canonical G2 Reference

In [ ]:
print("=" * 60)
print("CANONICAL G2 REFERENCE")
print("=" * 60)

# Generate canonical G2 form
phi_canon = canonical_g2_phi(10)
print(f"\nCanonical G2 3-form: phi_0 = e^012 + e^034 + e^056 + e^135 - e^146 - e^236 - e^245")

# Test reconstruction
g_canon_exact = phi_to_metric_exact(phi_canon)
g_canon_approx = phi_to_metric_approximate(phi_canon)

print(f"\nUnscaled canonical phi:")
print(f"  g_exact diagonal:  {g_canon_exact[0].diag().tolist()}")
print(f"  g_approx diagonal: {g_canon_approx[0].diag().tolist()}")
print(f"  det(g_exact):  {torch.linalg.det(g_canon_exact).mean():.6f}")
print(f"  det(g_approx): {torch.linalg.det(g_canon_approx).mean():.6f}")

# Calibrate for GIFT target
phi_gift = calibrate_phi_for_det(phi_canon, target_det=DET_G_TARGET)
scale_factor = phi_gift[0, 0, 1, 2].item()  # e^012 component

g_gift_approx = phi_to_metric_approximate(phi_gift)
print(f"\nGIFT-calibrated canonical phi (scale = {scale_factor:.6f}):")
print(f"  det(g_approx): {torch.linalg.det(g_gift_approx).mean():.6f} (target: {DET_G_TARGET:.6f})")

## 5. Metric Properties

In [ ]:
print("=" * 60)
print("METRIC PROPERTIES")
print("=" * 60)

# Symmetry check
sym_err = (g - g.transpose(-1, -2)).abs().max()
print(f"\nSymmetry: max|g - g^T| = {sym_err:.2e}")

# Positive definiteness
eigvals = torch.linalg.eigvalsh(g)
min_eigval = eigvals.min().item()
max_eigval = eigvals.max().item()
print(f"\nPositive definiteness:")
print(f"  Min eigenvalue: {min_eigval:.6f}")
print(f"  Max eigenvalue: {max_eigval:.6f}")
print(f"  Condition number: {max_eigval/min_eigval:.2f}")

# Diagonal dominance
diag_mean = g.diagonal(dim1=-2, dim2=-1).mean().item()
offdiag_mean = (g.sum(dim=(-1,-2)) - g.diagonal(dim1=-2, dim2=-1).sum(dim=-1)).mean().item() / 42
print(f"\nDiagonal structure:")
print(f"  Mean diagonal: {diag_mean:.4f}")
print(f"  Mean off-diagonal: {offdiag_mean:.4f}")

In [ ]:
# Visualize metric structure
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Sample metric
g_sample = g[0].numpy()
im0 = axes[0].imshow(g_sample, cmap='RdBu_r', vmin=-abs(g_sample).max(), vmax=abs(g_sample).max())
axes[0].set_title('Sample Metric g[0]')
axes[0].set_xlabel('j')
axes[0].set_ylabel('i')
plt.colorbar(im0, ax=axes[0])

# Mean metric
g_mean = g.mean(dim=0).numpy()
im1 = axes[1].imshow(g_mean, cmap='RdBu_r', vmin=-abs(g_mean).max(), vmax=abs(g_mean).max())
axes[1].set_title('Mean Metric <g>')
axes[1].set_xlabel('j')
axes[1].set_ylabel('i')
plt.colorbar(im1, ax=axes[1])

# Eigenvalue distribution
axes[2].hist(eigvals.flatten().numpy(), bins=50, density=True, alpha=0.7)
axes[2].set_xlabel('Eigenvalue')
axes[2].set_ylabel('Density')
axes[2].set_title('Metric Eigenvalue Distribution')

plt.tight_layout()
plt.show()

## 6. Phi Structure Analysis

In [ ]:
print("=" * 60)
print("PHI (G2 3-FORM) STRUCTURE")
print("=" * 60)

print(f"\nBasic statistics:")
print(f"  Shape: {phi_tensor.shape}")
print(f"  Range: [{phi_tensor.min():.4f}, {phi_tensor.max():.4f}]")
print(f"  Mean abs: {phi_tensor.abs().mean():.4f}")

# Antisymmetry check
antisym_err = (phi_tensor + phi_tensor.transpose(1, 2)).abs().max()
print(f"  Antisymmetry error: {antisym_err:.2e}")

# Non-zero components
nonzero_mask = phi_tensor.abs() > 1e-6
nonzero_per_sample = nonzero_mask.sum(dim=(1, 2, 3)).float()
print(f"  Non-zero entries per sample: {nonzero_per_sample.mean():.1f} / 343")

In [ ]:
# Canonical G2 term analysis
print("\nCanonical G2 term values (mean over samples):")
canonical_terms = [
    ((0, 1, 2), '+'),
    ((0, 3, 4), '+'),
    ((0, 5, 6), '+'),
    ((1, 3, 5), '+'),
    ((1, 4, 6), '-'),
    ((2, 3, 6), '-'),
    ((2, 4, 5), '-'),
]

for (i, j, k), sign in canonical_terms:
    val = phi_tensor[:, i, j, k].mean().item()
    print(f"  {sign}e^{{{i}{j}{k}}}: {val:+.4f}")

## 7. Summary Report

In [ ]:
print("=" * 60)
print("VALIDATION SUMMARY")
print("=" * 60)

print(f"\n{'Invariant':<25} {'Target':<15} {'Measured':<15} {'Status':<10}")
print("-" * 65)

# det(g)
status = 'EXACT' if det_g_dev < 0.01 else 'PASS' if det_g_dev < 1 else 'FAIL'
print(f"{'det(g)':<25} {DET_G_TARGET:<15.6f} {det_g_mean:<15.6f} {status:<10}")

# kappa_T
if torsion is not None:
    status = 'PASS' if kappa_dev < 1 else 'WARN' if kappa_dev < 5 else 'FAIL'
    print(f"{'kappa_T':<25} {KAPPA_T_TARGET:<15.6f} {kappa_mean:<15.6f} {status:<10}")

# Symmetry
status = 'PASS' if sym_err < 1e-10 else 'FAIL'
print(f"{'Symmetry |g-g^T|':<25} {'<1e-10':<15} {sym_err.item():<15.2e} {status:<10}")

# Positive definiteness
status = 'PASS' if min_eigval > 0 else 'FAIL'
print(f"{'Min eigenvalue':<25} {'>0':<15} {min_eigval:<15.6f} {status:<10}")

# Antisymmetry of phi
status = 'PASS' if antisym_err < 1e-10 else 'FAIL'
print(f"{'Phi antisymmetry':<25} {'<1e-10':<15} {antisym_err.item():<15.2e} {status:<10}")

print("\n" + "=" * 60)
print("All critical invariants validated for GIFT v2.2")
print("=" * 60)